In [1]:
import os
import cv2
import gc
import json
import numpy as np
import pandas as pd
import itertools
from tqdm.autonotebook import tqdm
import albumentations as A
import matplotlib.pyplot as plt

import torch
from torch import nn
import torch.nn.functional as F
import timm
from transformers import DistilBertModel, DistilBertConfig, DistilBertTokenizer


<ipython-input-1-aaa55f4caea2>:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.0 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForPreTraining

In [3]:
# Step 1: Mount Google Drive to access the dataset.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
import json
import cv2
import torch
import numpy as np
import pandas as pd
import albumentations as A
import torch.nn as nn
import torch.nn.functional as F
import timm
import itertools
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AutoConfig
import matplotlib.pyplot as plt

# Add get_lr function
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

class CFG:
    debug = False
    # Dataset paths
    image_path1 = '/content/drive/MyDrive/Bangla Image dataset with caption/Flickr8k_Dataset/Flicker8k_Dataset'
    captions_path1 = '/content/drive/MyDrive/Bangla Image dataset with caption/Flickr8k_Dataset'
    image_path2 = '/content/drive/MyDrive/Bangla Image dataset with caption/BNATURE/Pictures'
    captions_path2 = '/content/drive/MyDrive/Bangla Image dataset with caption/BNATURE/caption/captions.json'
    image_path3 = '/content/drive/MyDrive/Bangla Image dataset with caption/Bangla Lekha 2.0/images'
    captions_path3 = '/content/drive/MyDrive/Bangla Image dataset with caption/Bangla Lekha 2.0/captions.json'

    # Training parameters
    batch_size = 32  # Reduced batch size
    gradient_accumulation_steps = 4  # Increased accumulation steps
    num_workers = 2
    pin_memory = True
    # Mixed precision settings
    mixed_precision = True

    # Model settings

    attention_dropout_prob = 0.1

    # Optimizer settings
    weight_decay = 0.01
    max_grad_norm = 1.0

    # Learning rates
    image_encoder_lr = 1e-5
    text_encoder_lr = 1e-5
    head_lr = 1e-4
    prefetch_factor = 2
    patience = 1
    hidden_dropout_prob=0.1
    factor = 0.8
    epochs = 2
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Model parameters
    model_name = 'resnet50'
    image_embedding = 2048
    text_encoder_model = "csebuetnlp/banglabert"
    text_embedding = 768
    text_tokenizer = "csebuetnlp/banglabert"
    max_length = 200
    pretrained = True
    trainable = True
    temperature = 1.0
    size = 224
    num_projection_layers = 1
    projection_dim = 256
    dropout = 0.1

In [5]:
def load_captions():
    """Load captions with improved validation"""
    captions_list = []

    # Load Flickr8k dataset
    try:
        with open(os.path.join(CFG.captions_path1, 'BAN-Cap_captiondata.json'), 'r', encoding='utf-8') as f:
            captions_data1 = json.load(f)

        for entry in captions_data1:
            if isinstance(entry, dict) and 'caption_id' in entry and 'bengali_caption' in entry:
                filename = str(entry['caption_id']).split('#')[0]
                caption = str(entry['bengali_caption'])
                if caption and filename:
                    captions_list.append({
                        "image": filename.strip(),
                        "caption": caption.strip()
                    })
    except Exception as e:
        print(f"Error loading Flickr8k dataset: {str(e)}")

    # Load BNATURE dataset
    try:
        with open(CFG.captions_path2, 'r', encoding='utf-8') as f:
            captions_data2 = json.load(f)

        for entry in captions_data2:
            if isinstance(entry, dict) and 'caption_id' in entry and 'bengali_caption' in entry:
                filename = str(entry['caption_id'])
                caption = str(entry['bengali_caption'])
                if caption and filename:
                    captions_list.append({
                        "image": filename.strip(),
                        "caption": caption.strip()
                    })
    except Exception as e:
        print(f"Error loading BNATURE dataset: {str(e)}")

    # Load Bangla Lekha dataset with improved handling
    try:
        with open(CFG.captions_path3, 'r', encoding='utf-8') as f:
            captions_data3 = json.load(f)

        if isinstance(captions_data3, list):
            for entry in captions_data3:
                if isinstance(entry, dict) and 'filename' in entry and 'caption' in entry:
                    filename = str(entry['filename'])
                    caption = str(entry['caption'])
                    if caption and filename:
                        captions_list.append({
                            "image": filename.strip(),
                            "caption": caption.strip()
                        })
    except Exception as e:
        print(f"Error loading Bangla Lekha dataset: {str(e)}")

    df = pd.DataFrame(captions_list)
    df = df.dropna()
    df = df.drop_duplicates()
    df['id'] = df.index // 5

    print(f"Loaded {len(df)} valid caption entries")
    return df

In [6]:
class CLIPDataset(torch.utils.data.Dataset):
    def __init__(self, image_filenames, captions, tokenizer, transforms):
        self.image_filenames = image_filenames
        self.captions = list(captions)
        self.encoded_captions = tokenizer(
            list(captions),
            padding='max_length',
            truncation=True,
            max_length=CFG.max_length,
            return_tensors='pt'
        )
        self.transforms = transforms

        self.valid_indices = []
        for idx in range(len(self.image_filenames)):
            try:
                image_found = False
                for path in [CFG.image_path1, CFG.image_path2, CFG.image_path3]:
                    if os.path.exists(os.path.join(path, self.image_filenames[idx])):
                        image_found = True
                        break

                if image_found:
                    self.valid_indices.append(idx)
            except Exception as e:
                continue

        print(f"Found {len(self.valid_indices)} valid images out of {len(image_filenames)}")

    def __getitem__(self, idx):
        try:
            actual_idx = self.valid_indices[idx]

            item = {
                'input_ids': self.encoded_captions['input_ids'][actual_idx],
                'attention_mask': self.encoded_captions['attention_mask'][actual_idx],
            }

            image_path = None
            for path in [CFG.image_path1, CFG.image_path2, CFG.image_path3]:
                if os.path.exists(os.path.join(path, self.image_filenames[actual_idx])):
                    image_path = path
                    break

            if image_path is None:
                raise FileNotFoundError(f"Image {self.image_filenames[actual_idx]} not found in any path")

            image = cv2.imread(os.path.join(image_path, self.image_filenames[actual_idx]))
            if image is None:
                raise ValueError(f"Failed to load image: {self.image_filenames[actual_idx]}")

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = self.transforms(image=image)['image']
            item['image'] = torch.tensor(image).permute(2, 0, 1).float()
            item['caption'] = self.captions[actual_idx]

            return item
        except Exception as e:
            print(f"Error processing item {idx}: {str(e)}")
            raise e

    def __len__(self):
        return len(self.valid_indices)

In [7]:


def build_loaders(dataframe, tokenizer, mode):
    """
    Build data loaders with error handling
    """
    transforms = get_transforms(mode=mode)

    try:
        dataset = CLIPDataset(
            image_filenames=dataframe["image"].values,
            captions=dataframe["caption"].values,
            tokenizer=tokenizer,
            transforms=transforms
        )

        # Custom collate function to handle potential None values
        def collate_fn(batch):
            # Filter out None values
            batch = [item for item in batch if item is not None]
            if len(batch) == 0:
                raise RuntimeError("Empty batch after filtering")

            return {
                'image': torch.stack([item['image'] for item in batch]),
                'input_ids': torch.stack([item['input_ids'] for item in batch]),
                'attention_mask': torch.stack([item['attention_mask'] for item in batch]),
                'caption': [item['caption'] for item in batch]
            }

        dataloader = torch.utils.data.DataLoader(
            dataset,
            batch_size=CFG.batch_size,
            num_workers=CFG.num_workers,
            shuffle=True if mode == "train" else False,
            collate_fn=collate_fn,
            drop_last=True  # Drop incomplete batches
        )

        return dataloader

    except Exception as e:
        print(f"Error building dataloader: {str(e)}")
        raise e

In [8]:

class ImageEncoder(nn.Module):
    def __init__(
        self, model_name=CFG.model_name, pretrained=CFG.pretrained, trainable=CFG.trainable
    ):
        super().__init__()
        self.model = timm.create_model(
            model_name, pretrained, num_classes=0, global_pool="avg"
        )
        for p in self.model.parameters():
            p.requires_grad = trainable

    def forward(self, x):
        return self.model(x)

from transformers import AutoTokenizer, AutoModel, AutoConfig

class TextEncoder(nn.Module):
    def __init__(self, model_name=CFG.text_encoder_model, pretrained=CFG.pretrained, trainable=CFG.trainable):
        super().__init__()
        if pretrained:
            self.model = AutoModel.from_pretrained(model_name)
        else:
            self.model = AutoModel(config=AutoConfig.from_pretrained(model_name))

        for p in self.model.parameters():
            p.requires_grad = trainable
        self.target_token_idx = 0

    def forward(self, input_ids, attention_mask):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = output.last_hidden_state
        return last_hidden_state[:, self.target_token_idx, :]

class ProjectionHead(nn.Module):
    def __init__(
        self,
        embedding_dim,
        projection_dim=CFG.projection_dim,
        dropout=CFG.dropout
    ):
        super().__init__()
        self.projection = nn.Linear(embedding_dim, projection_dim)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(projection_dim, projection_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(projection_dim)

    def forward(self, x):
        projected = self.projection(x)
        x = self.gelu(projected)
        x = self.fc(x)
        x = self.dropout(x)
        x = x + projected
        x = self.layer_norm(x)
        return x


In [9]:
class CLIPModel(nn.Module):
    def __init__(
        self,
        temperature=CFG.temperature,
        image_embedding=CFG.image_embedding,
        text_embedding=CFG.text_embedding,
    ):
        super().__init__()
        self.image_encoder = ImageEncoder()
        self.text_encoder = TextEncoder()
        self.image_projection = ProjectionHead(embedding_dim=image_embedding)
        self.text_projection = ProjectionHead(embedding_dim=text_embedding)
        self.temperature = temperature

    def forward(self, batch):
        image_features = self.image_encoder(batch["image"])
        text_features = self.text_encoder(
            input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
        )
        image_embeddings = self.image_projection(image_features)
        text_embeddings = self.text_projection(text_features)

        logits = (text_embeddings @ image_embeddings.T) / self.temperature
        images_similarity = image_embeddings @ image_embeddings.T
        texts_similarity = text_embeddings @ text_embeddings.T
        targets = F.softmax(
            (images_similarity + texts_similarity) / 2 * self.temperature, dim=-1
        )
        texts_loss = F.cross_entropy(logits, targets, reduction='none')
        images_loss = F.cross_entropy(logits.T, targets.T, reduction='none')
        loss = (images_loss + texts_loss) / 2.0
        return loss.mean()


In [10]:

def cross_entropy(preds, targets, reduction='none'):
    log_softmax = nn.LogSoftmax(dim=-1)
    loss = (-targets * log_softmax(preds)).sum(1)
    if reduction == "none":
        return loss
    elif reduction == "mean":
        return loss.mean()

In [11]:

def get_transforms(mode="train"):
    if mode == "train":
        return A.Compose(
            [
                A.Resize(CFG.size, CFG.size, always_apply=True),
                A.Normalize(max_pixel_value=255.0, always_apply=True),
            ]
        )
    else:
        return A.Compose(
            [
                A.Resize(CFG.size, CFG.size, always_apply=True),
                A.Normalize(max_pixel_value=255.0, always_apply=True),
            ]
        )


In [12]:

def make_train_valid_dfs():
    dataframe = load_captions()
    max_id = dataframe["id"].max() + 1 if not CFG.debug else 100
    image_ids = np.arange(0, max_id)
    np.random.seed(42)
    valid_ids = np.random.choice(
        image_ids, size=int(0.2 * len(image_ids)), replace=False
    )
    train_ids = [id_ for id_ in image_ids if id_ not in valid_ids]
    train_dataframe = dataframe[dataframe["id"].isin(train_ids)].reset_index(drop=True)
    valid_dataframe = dataframe[dataframe["id"].isin(valid_ids)].reset_index(drop=True)
    return train_dataframe, valid_dataframe

In [13]:

def build_loaders(dataframe, tokenizer, mode):
    transforms = get_transforms(mode=mode)
    dataset = CLIPDataset(
        dataframe["image"].values,
        dataframe["caption"].values,
        tokenizer=tokenizer,
        transforms=transforms,
    )
    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=CFG.batch_size,
        num_workers=CFG.num_workers,
        shuffle=True if mode == "train" else False,
    )
    return dataloader



In [14]:
class AvgMeter:
    """
    Computes and stores the average and current value
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [15]:
import torch
import itertools
from tqdm import tqdm
from transformers import AutoTokenizer
import matplotlib.pyplot as plt
import numpy as np

In [16]:
import torch
import itertools
from tqdm import tqdm
from transformers import AutoTokenizer, get_linear_schedule_with_warmup
import matplotlib.pyplot as plt
import numpy as np
import gc

# Memory optimizations
torch.cuda.empty_cache()
torch.backends.cudnn.benchmark = True
if torch.cuda.is_available():
    torch.backends.cuda.enable_mem_efficient_sdp(True)

# Configure PyTorch memory allocator
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128,garbage_collection_threshold:0.8,expandable_segments:True'


In [ ]:
import torch
import itertools
from tqdm import tqdm
from transformers import AutoTokenizer, get_linear_schedule_with_warmup
import matplotlib.pyplot as plt
import numpy as np

# Set memory and speed optimizations
torch.cuda.empty_cache()
torch.backends.cudnn.benchmark = True
if torch.cuda.is_available():
    torch.backends.cuda.enable_mem_efficient_sdp(True)

# Configure PyTorch memory allocator
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512,expandable_segments:True'

def compute_cosine_similarity(embeddings_a, embeddings_b, temperature=0.07):
    # Process in chunks to save memory
    chunk_size = 256
    num_chunks = (embeddings_a.size(0) + chunk_size - 1) // chunk_size
    similarity_chunks = []

    for i in range(num_chunks):
        start_idx = i * chunk_size
        end_idx = min((i + 1) * chunk_size, embeddings_a.size(0))
        chunk_a = embeddings_a[start_idx:end_idx]

        # Normalize chunks
        chunk_a = torch.nn.functional.normalize(chunk_a, p=2, dim=-1)
        chunk_b = torch.nn.functional.normalize(embeddings_b, p=2, dim=-1)

        # Compute similarity for chunk
        chunk_sim = torch.mm(chunk_a, chunk_b.t()) / temperature
        similarity_chunks.append(chunk_sim)

    return torch.cat(similarity_chunks, dim=0)

def precision_recall(similarity_matrix, k=5):
    num_samples = similarity_matrix.size(0)
    relevant_items = torch.eye(num_samples, device=similarity_matrix.device)
    top_k_indices = similarity_matrix.topk(k, dim=-1).indices

    precision = 0
    recall = 0
    for i in range(num_samples):
        matches = relevant_items[i][top_k_indices[i]].sum().item()
        precision += matches / k
        recall += matches / relevant_items[i].sum().item()

    precision /= num_samples
    recall /= num_samples
    return precision, recall

def train(model, train_loader, optimizer, lr_scheduler, scaler, epoch):
    model.train()
    loss_meter = AvgMeter()

    # Process fewer batches for validation
    eval_steps = min(100, len(train_loader))

    image_embeddings_list = []
    text_embeddings_list = []

    for batch_idx, batch in enumerate(tqdm(train_loader, total=len(train_loader))):
        # Move to GPU and handle memory
        batch = {k: v.to(CFG.device, non_blocking=True) for k, v in batch.items() if k != "caption"}

        # Clear memory
        if batch_idx % 50 == 0:
            torch.cuda.empty_cache()

        # Mixed precision training
        with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
            loss = model(batch)
            loss = loss / CFG.gradient_accumulation_steps

        scaler.scale(loss).backward()

        if (batch_idx + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            lr_scheduler.step()

        # Collect embeddings for evaluation
        if batch_idx < eval_steps:
            with torch.no_grad():
                image_features = model.image_encoder(batch["image"])
                text_features = model.text_encoder(
                    input_ids=batch["input_ids"],
                    attention_mask=batch["attention_mask"]
                )
                image_embeddings = model.image_projection(image_features)
                text_embeddings = model.text_projection(text_features)

                image_embeddings_list.append(image_embeddings.detach().cpu())
                text_embeddings_list.append(text_embeddings.detach().cpu())

        loss_meter.update(loss.item() * CFG.gradient_accumulation_steps, batch["image"].size(0))

        # Early stopping for first epoch
        if epoch == 0 and batch_idx >= 200:
            break

    # Calculate metrics
    precision, recall = 0, 0
    if image_embeddings_list:
        image_embeddings = torch.cat(image_embeddings_list, dim=0)
        text_embeddings = torch.cat(text_embeddings_list, dim=0)

        similarity_matrix = compute_cosine_similarity(image_embeddings, text_embeddings)
        precision, recall = precision_recall(similarity_matrix, k=5)

        del image_embeddings, text_embeddings, similarity_matrix
        torch.cuda.empty_cache()

    return loss_meter, precision, recall

def validate(model, valid_loader):
    model.eval()
    loss_meter = AvgMeter()

    # Reduce validation samples
    max_val_samples = 1000

    image_embeddings_list = []
    text_embeddings_list = []

    with torch.no_grad():
        for batch_idx, batch in enumerate(valid_loader):
            if batch_idx >= max_val_samples:
                break

            batch = {k: v.to(CFG.device, non_blocking=True) for k, v in batch.items() if k != "caption"}

            # Process in half precision
            with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                image_features = model.image_encoder(batch["image"])
                text_features = model.text_encoder(
                    input_ids=batch["input_ids"],
                    attention_mask=batch["attention_mask"]
                )
                image_embeddings = model.image_projection(image_features)
                text_embeddings = model.text_projection(text_features)

                similarity = compute_cosine_similarity(image_embeddings, text_embeddings)
                loss = model.compute_loss(similarity)

            image_embeddings_list.append(image_embeddings.cpu())
            text_embeddings_list.append(text_embeddings.cpu())

            loss_meter.update(loss.item(), batch["image"].size(0))

            # Clear memory periodically
            if batch_idx % 50 == 0:
                torch.cuda.empty_cache()

    precision, recall = 0, 0
    if image_embeddings_list:
        image_embeddings = torch.cat(image_embeddings_list, dim=0)
        text_embeddings = torch.cat(text_embeddings_list, dim=0)

        similarity_matrix = compute_cosine_similarity(image_embeddings, text_embeddings)
        precision, recall = precision_recall(similarity_matrix, k=5)

        del image_embeddings, text_embeddings, similarity_matrix
        torch.cuda.empty_cache()

    return loss_meter, precision, recall

def main():
    print("Starting memory-optimized training...")

    # Initialize lists to store metrics for plotting
    train_losses = []
    val_losses = []
    train_precisions = []
    val_precisions = []
    train_recalls = []
    val_recalls = []

    # Load data with reduced batch size
    train_df, valid_df = make_train_valid_dfs()
    tokenizer = AutoTokenizer.from_pretrained(CFG.text_tokenizer)

    train_loader = build_loaders(train_df, tokenizer, mode="train")
    valid_loader = build_loaders(valid_df, tokenizer, mode="valid")

    # Remove incorrect arguments and use only the ones defined in CLIPModel __init__
    model = CLIPModel(
        temperature=CFG.temperature,
        image_embedding=CFG.image_embedding,
        text_embedding=CFG.text_embedding
    ).to(CFG.device)

    # Load checkpoint
    checkpoint_path = "/content/drive/MyDrive/Bangla Image dataset with caption/banglaclipcombinedfinal.pt"
    checkpoint = torch.load(checkpoint_path, map_location=CFG.device)
    model.load_state_dict(checkpoint)
    del checkpoint
    torch.cuda.empty_cache()

    # Optimizer setup with reduced learning rates
    params = [
        {"params": model.image_encoder.parameters(), "lr": CFG.image_encoder_lr},
        {"params": model.text_encoder.parameters(), "lr": CFG.text_encoder_lr},
        {"params": itertools.chain(
            model.image_projection.parameters(),
            model.text_projection.parameters()
        ), "lr": CFG.head_lr}
    ]

    optimizer = torch.optim.AdamW(params, weight_decay=CFG.weight_decay)
    scaler = torch.amp.GradScaler()

    # Scheduler with reduced warmup
    num_training_steps = len(train_loader) * CFG.epochs
    num_warmup_steps = num_training_steps // 20  # 5% warmup

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps
    )

    best_val_loss = float('inf')
    patience = 2
    early_stopping_counter = 0

    for epoch in range(CFG.epochs):
        print(f"\nEpoch {epoch + 1}")

        train_loss, train_prec, train_rec = train(
            model, train_loader, optimizer, scheduler, scaler, epoch
        )

        val_loss, val_prec, val_rec = validate(model, valid_loader)

        # Append metrics for plotting
        train_losses.append(train_loss.avg)
        val_losses.append(val_loss.avg)
        train_precisions.append(train_prec)
        val_precisions.append(val_prec)
        train_recalls.append(train_rec)
        val_recalls.append(val_rec)

        print(f"Train Loss: {train_loss.avg:.4f}, P@5: {train_prec:.4f}, R@5: {train_rec:.4f}")
        print(f"Val Loss: {val_loss.avg:.4f}, P@5: {val_prec:.4f}, R@5: {val_rec:.4f}")

        # Save best model and handle early stopping
        if val_loss.avg < best_val_loss:
            best_val_loss = val_loss.avg
            torch.save(model.state_dict(), f"best_model_epoch_{epoch+1}.pth")
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= patience:
                print("Early stopping triggered")
                break

        # Clear memory between epochs
        torch.cuda.empty_cache()
        gc.collect()

        # Plot training progress
        plt.figure(figsize=(15, 5))
        epochs_range = range(1, epoch + 2)

        plt.subplot(1, 3, 1)
        plt.plot(epochs_range, train_losses, label="Train Loss", marker="o", color="blue")
        plt.plot(epochs_range, val_losses, label="Val Loss", marker="x", color="green")
        plt.title("Loss")
        plt.legend()

        plt.subplot(1, 3, 2)
        plt.plot(epochs_range, train_precisions, label="Train", marker="o", color="orange")
        plt.plot(epochs_range, val_precisions, label="Val", marker="x", color="red")
        plt.title("Precision@5")
        plt.legend()

        plt.subplot(1, 3, 3)
        plt.plot(epochs_range, train_recalls, label="Train", marker="o", color="purple")
        plt.plot(epochs_range, val_recalls, label="Val", marker="x", color="brown")
        plt.title("Recall@5")
        plt.legend()

        plt.tight_layout()
        plt.show()
        plt.close()

        # Clear memory between epochs
        torch.cuda.empty_cache()
        gc.collect()

if __name__ == "__main__":
    main()

Starting memory-optimized training...
Loaded 88641 valid caption entries


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Found 70919 valid images out of 70919
Found 17722 valid images out of 17722


<ipython-input-17-b14acf9ed9e6>:201: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=CFG.device)



Epoch 1


  1%|          | 12/2217 [02:36<7:04:35, 11.55s/it]

In [ ]:
import torch

# Define the Google Drive path for saving the model
save_path = "/content/drive/MyDrive/Bangla Image dataset with caption/banglaclipcombinedfinalnowok.pt"

# Define your model class (ensure this matches the architecture used during training)
model = CLIPModel().to(CFG.device)  # Replace CLIPModel with your actual model class

# Load the trained model from the saved state dictionary
checkpoint_path = "/content/best_model.pth"
model.load_state_dict(torch.load(checkpoint_path))

# Save the model's state dictionary to the specified Google Drive path
torch.save(model.state_dict(), save_path)

print(f"Model saved successfully at: {save_path}")


#Interface

In [ ]:
def get_image_embeddings(dataframe, model_path):
    """
    Loads a model and generates image embeddings for the provided dataset.

    Parameters:
    - dataframe: DataFrame containing image file names.
    - model_path: Path to the saved model file.

    Returns:
    - model: Loaded CLIPModel instance.
    - image_embeddings: Tensor of image embeddings for the dataset.
    """
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(CFG.text_tokenizer)
    model = CLIPModel().to(CFG.device)
    model.load_state_dict(torch.load(model_path, map_location=CFG.device))
    model.eval()

    # Prepare data loader
    transforms = get_transforms(mode="valid")
    dataset = CLIPDataset(
        dataframe["image"].values,
        dataframe["caption"].values,
        tokenizer=tokenizer,
        transforms=transforms,
    )
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=CFG.batch_size, num_workers=CFG.num_workers, shuffle=False
    )

    # Generate image embeddings
    image_embeddings_list = []
    with torch.no_grad():
        for batch in tqdm(dataloader, colour="yellow", desc="Generating image embeddings"):
            images = batch["image"].to(CFG.device)
            image_features = model.image_encoder(images)
            image_embeddings = model.image_projection(image_features)
            image_embeddings_list.append(image_embeddings)

    # Concatenate all embeddings
    image_embeddings = torch.cat(image_embeddings_list, dim=0)
    return model, image_embeddings


# Interface Function to Use find_matches
if __name__ == "__main__":
    # Ensure the validation DataFrame and model are ready
    _, valid_df = make_train_valid_dfs()

    # Generate the model and image embeddings
    model, image_embeddings = get_image_embeddings(valid_df, "best_model.pth")

    # Define the Bangla text query and display matches
    query = "কক্সবাজারের সমুদ্র সৈকত"  # Bangla query for "cycle"
    find_matches(
        model=model,
        image_embeddings=image_embeddings,
        query=query,
        image_filenames=valid_df["image"].values,
        n=9,
    )
